In [1]:
from dist_yolo.yolo import YOLO
import tensorflow as tf
import numpy as np
import cv2 
from math import ceil

In [9]:
vid_path = r"C:\Users\AbdElRahman_ElGharib\Downloads\Compressed\Test Video.mp4"
out_path = r"C:\Users\AbdElRahman_ElGharib\Desktop\output_video.avi"
chkpt = r"C:\Users\AbdElRahman_ElGharib\Downloads\checkpoint\checkpoint"

In [3]:
BOUNDING_BOX_FORMAT = 'xyxy'
SPLIT_RATIO = 0.2
EPOCH = 50
BATCH_SIZE = 32
LEARNING_RATE = 5e-3
WEIGHT_DECAY = 3e-4
DECAY_EPOCHS = 5
GLOBAL_CLIPNORM = 10.0
IMAGE_SIZE = 640
D = 0.35
W = 0.25
R = 2.0
NC = 8

m = 7481

In [4]:
yolo = YOLO(NC, D, W, R, iou_threshold=0.5)

yolo.compile(
    box_loss_weight=7.5,
    classification_loss_weight=0.5,
    distance_loss_weight=0.01,
    optimizer=tf.keras.optimizers.experimental.SGD(
        learning_rate=tf.keras.optimizers.schedules.CosineDecayRestarts(
            initial_learning_rate=LEARNING_RATE,
            first_decay_steps=(DECAY_EPOCHS * int(ceil((m-int(m*SPLIT_RATIO))/BATCH_SIZE))),
            alpha=1e-5,
        ),
        momentum=0.937,
        weight_decay=WEIGHT_DECAY,
        global_clipnorm=GLOBAL_CLIPNORM,
        use_ema=True,
        ema_momentum=0.9999
    )
)

yolo.load_weights(chkpt)

In [10]:
video_capture = cv2.VideoCapture(vid_path)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(
    out_path,
    fourcc,
    30,
    (IMAGE_SIZE, IMAGE_SIZE)
)

num_frame = 0

while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break
    
    frame = cv2.resize(
        frame,
        (IMAGE_SIZE, IMAGE_SIZE)
    )
    
    pred = yolo.predict(
        np.expand_dims(
            np.array(
                cv2.cvtColor(
                    frame,
                    cv2.COLOR_BGR2RGB
                ),
                dtype=np.uint8
                ),
            axis=0
        )
    )
    
    boxes = []
    classes = []
    dist = []
    conf = []
    
    for i in range(pred['num_detections'][0]):
        boxes.append(list(map(int,pred["boxes"][0][i].tolist())))
        classes.append(pred["classes"][0][i].tolist())
        dist.append(pred['distances'][0][i].tolist())
        conf.append(pred["confidence"][0][i].tolist())
    
    # print(boxes)
    # print(classes)
    # print(dist)
    # print(conf)
    
    for i in range(pred['num_detections'][0]):
        frame = cv2.putText(
            cv2.rectangle(
                frame,
                (boxes[i][0],boxes[i][1]),
                (boxes[i][2],boxes[i][3]),
                (0,0,255),
                2
            ),
            str(classes[i])+"|"+str(round(dist[i], 2))+"|"+str(round(conf[i], 3)),
            (boxes[i][0],boxes[i][1]-3),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0,0,255),
            2
        )
    
    num_frame += 1
    print('Frame:\t'+str(num_frame))
    
    output_video.write(frame)
    # cv2.imshow('test', frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

video_capture.release()
output_video.release()

1/1 [==============================] - 0s 364ms/step
Frame:	1
1/1 [==============================] - 0s 425ms/step
Frame:	2
1/1 [==============================] - 0s 368ms/step
Frame:	3
1/1 [==============================] - 0s 303ms/step
Frame:	4
1/1 [==============================] - 0s 360ms/step
Frame:	5
1/1 [==============================] - 0s 369ms/step
Frame:	6
1/1 [==============================] - 0s 385ms/step
Frame:	7
1/1 [==============================] - 0s 309ms/step
Frame:	8
1/1 [==============================] - 0s 304ms/step
Frame:	9
1/1 [==============================] - 0s 311ms/step
Frame:	10
1/1 [==============================] - 0s 304ms/step
Frame:	11
1/1 [==============================] - 0s 306ms/step
Frame:	12
1/1 [==============================] - 0s 291ms/step
Frame:	13
1/1 [==============================] - 0s 299ms/step
Frame:	14
1/1 [==============================] - 0s 306ms/step
Frame:	15
1/1 [==============================] - 0s 303ms/step
Frame:	16
1